In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform,randint
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/preprocess_data.csv',sep = ',')

In [3]:
pd.options.display.max_columns = None
data.head(5)

,Tournament,Home Team,Away Team,Match Date,Winner Team,Winner Home Or Away,Phase,Final Score,Extra Periods,Team,Opponent Team,Team Points,Team Two Pointers Made,Team Two Pointers Attempted,Team Two Pointers Percenage,Team Three Pointers Made,Team Three Pointers Attempted,Team Three Pointers Percenage,Team Free Throws Made,Team Free Throws Attempted,Team Free Throws Percenage,Team Offensive Rebounds,Team Defensive Rebounds,Team Total Rebounds,Team Assists,Team Steals,Team Turnovers,Team Blocks,Team Blocks Against,Team Personal Fouls,Team Fouls Received,Team Performance Index,Opponent Points,Opponent Two Pointers Made,Opponent Two Pointers Attempted,Opponent Two Pointers Percenage,Opponent Three Pointers Made,Opponent Three Pointers Attempted,Opponent Three Pointers Percenage,Opponent Free Throws Made,Opponent Free Throws Attempted,Opponent Free Throws Percenage,Opponent Offensive Rebounds,Opponent Defensive Rebounds,Opponent Total Rebounds,Opponent Assists,Opponent Steals,Opponent Turnovers,Opponent Blocks,Opponent Blocks Against,Opponent Personal Fouls,Opponent Fouls Received,Opponent Performance Index,Team Result,teamFGA,teamFGM,teamTREB%,teamASST%,teamTS%,teamEFG%,teamOREB%,teamDREB%,teamTO%,teamPoss,teamSTL%,teamBLK%,teamBLKR,teamPPS,teamPlay%,teamAR,teamAST/TO,teamSTL/TO,teamFIC,teamOrtg,opptFGA,opptPoss,teamDrtg,teamEDiff,team_Game_Score,opptFGM,opptTREB%,opptASST%,opptTS%,opptEFG%,opptOREB%,opptDREB%,opptTO%,opptSTL%,opptBLK%,opptBLKR,opptPPS,opptFIC,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,oppt_Game_Score,Points difference,Finals,Last 32,Playoffs,Quarter-Finals,Regular Season,Semifinals,Top 16
0,Basket League,KAOD,Aris,2013-10-12,Aris,Away,Regular Season,58-66,0.0,KAOD,Aris,58.0,20.0,43.0,0.46512,1.0,10.0,0.10000,15.0,26.0,0.57692,12.0,22.0,34.0,6.0,6.0,17.0,3.0,3.0,22.0,22.0,40.0,66.0,15.0,33.0,0.45455,9.0,23.0,0.39130,9.0,19.0,0.47368,10.0,18.0,28.0,11.0,8.0,14.0,3.0,3.0,22.0,22.0,57.0,0.0,53.0,21.0,54.838710,28.571429,45.003104,40.566038,40.000000,68.750000,20.874263,69.44,8.640553,4.320276,9.090909,1.094340,36.206897,9.293680,0.352941,0.352941,24.000,83.525346,56.0,68.36,96.547689,-13.022343,26.4,24.0,45.161290,45.833333,51.274083,50.892857,31.250000,60.000000,17.866258,11.702750,4.388531,6.976744,1.178571,37.375,96.547689,83.525346,13.022343,40.000000,15.143172,0.785714,0.571429,39.8,-8.0,0,0,0,0,1,0,0
1,Basket League,PAOK,Kolossos Rhodes,2013-10-12,PAOK,Home,Regular Season,88-81,0.0,PAOK,Kolossos Rhodes,88.0,17.0,35.0,0.48571,10.0,26.0,0.38462,24.0,26.0,0.92308,9.0,22.0,31.0,22.0,7.0,18.0,3.0,1.0,23.0,23.0,96.0,81.0,26.0,42.0,0.61905,5.0,16.0,0.31250,14.0,21.0,0.66667,6.0,25.0,31.0,5.0,8.0,18.0,1.0,3.0,24.0,22.0,69.0,1.0,61.0,27.0,50.000000,81.481481,60.739923,52.459016,26.470588,78.571429,19.902698,81.44,8.595285,3.683694,7.142857,1.442623,38.571429,24.564538,1.222222,0.388889,60.500,108.055010,58.0,79.24,102.221100,5.833909,65.5,31.0,50.000000,16.129032,60.232005,57.758621,21.428571,73.529412,21.116847,10.095911,1.261989,2.857143,1.396552,38.375,102.221100,108.055010,-5.833909,44.285714,6.967670,0.277778,0.444444,46.3,7.0,0,0,0,0,1,0,0
2,Liga ACB,Bilbao,Zaragoza,2013-10-12,Zaragoza,Away,Regular Season,77-86,0.0,Bilbao,Zaragoza,77.0,24.0,46.0,0.52000,5.0,12.0,0.42000,14.0,20.0,0.70000,13.0,19.0,32.0,15.0,7.0,12.0,1.0,1.0,18.0,23.0,89.0,86.0,25.0,39.0,0.64000,10.0,23.0,0.43000,6.0,10.0,0.60000,9.0,20.0,29.0,16.0,10.0,10.0,1.0,1.0,23.0,18.0,95.0,0.0,58.0,29.0,52.459016,51.724138,57.634731,54.310345,39.393939,67.857143,15.228426,65.80,10.638298,1.519757,2.564103,1.327586,50.877193,19.685039,1.250000,0.583333,55.250,117.021277,62.0,67.40,127.596439,-10.575163,59.4,35.0,47.540984,45.714286,64.759036,64.516129,32.142857,60.606061,13.089005,14.836795,1.483680,2.173913,1.387097,65.250,127.596439,117.021277,10.575163,55.555556,19.138756,1.600000,1.000000,70.0,-9.0,0,0,0,0,1,0,0
3,Basket League,Trikala,Kifisia,2013-10-12,Kifisia,Away,Regular Season,64-69,0.0,Trikala,Kifisia,64.0,21.0,41.0,0.51220,

In [4]:
data["Tournament"].unique()

array(['Basket League', 'Liga ACB', 'Eurocup', 'Euroleague'], dtype=object)

# Teams as Dummy Variables

In [5]:
baseline_data = data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]]
baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Basket League,2013-10-12,KAOD,Aris,0.0,Aris,Regular Season
1,Basket League,2013-10-12,PAOK,Kolossos Rhodes,1.0,PAOK,Regular Season
2,Liga ACB,2013-10-12,Bilbao,Zaragoza,0.0,Zaragoza,Regular Season
3,Basket League,2013-10-12,Trikala,Kifisia,0.0,Kifisia,Regular Season
4,Liga ACB,2013-10-12,Tenerife,Obradoiro,1.0,Tenerife,Regular Season


## Greek Basket League 

In [6]:
greek_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Basket League'].reset_index(drop = True)
greek_baseline_data['Phase'].unique()

array(['Regular Season', 'Playoffs'], dtype=object)

In [7]:
greece_home = pd.get_dummies(greek_baseline_data["Home Team"]).reset_index(drop = True)
greece_away = -1*pd.get_dummies(greek_baseline_data["Away Team"]).reset_index(drop = True)
greece_x = pd.concat([greece_home,greece_away],axis=1).groupby(level=0, axis=1).sum()
greece_y = greek_baseline_data["Team Result"]

In [8]:
x_structure_greece = greece_x.iloc[:len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_greece = greece_y.iloc[:len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])]

start = len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])
end = len(pd.concat([greek_baseline_data[(greek_baseline_data["Match Date"]<"2017-8-1")],
                     greek_baseline_data[(greek_baseline_data["Match Date"]>"2017-8-1")&(
                         greek_baseline_data['Phase'].isin(['Regular Season']))]]))

x_greece_17_18 = greece_x.iloc[start:end]
y_greece_17_18 = greece_y.iloc[start:end]

x_train_greece = greece_x[:start + int(len(x_greece_17_18)/2)]
y_train_greece = greece_y.iloc[:start + int(len(x_greece_17_18)/2)]

x_test_greece = x_greece_17_18[int(len(x_greece_17_18)/2):]
y_test_greece = y_greece_17_18.iloc[int(len(y_greece_17_18)/2):]

###  Logistic Regression

In [9]:
lgr = LogisticRegression(C = 0.3839820885947254,penalty = 'l1', solver= 'liblinear').fit(x_train_greece, y_train_greece)
y_pre_proba_lgr=lgr.predict_proba(x_test_greece)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_greece),3))
print()
print("Test F1-score",round(f1_score(y_pre_lgr,y_test_greece),3))


Logistic Regression

Test BS: 0.15

Test Accuracy: 0.78

Test F1-score 0.841


In [10]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],greece_x.columns.values):
    coef_dict[feat] = coef
coef1=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef1=coef1.iloc[(-coef1[0].abs()).argsort()].reset_index()

In [11]:
sorted_coef1

,index,0
0,Olympiacos,3.529917
1,Panathinaikos,3.362932
2,AEK,1.337522
3,Aris,0.931565
4,PAOK,0.928383
5,Doxa Lefkadas,-0.749338
6,Panelefsiniakos,-0.528429
7,Kavala,-0.504153
8,Kolossos Rhodes,0.279209
9,Ikaros,-0.221916


In [12]:
lgr.intercept_

array([0.82749472])

## Random Forest Classifier

In [13]:
rfc = RandomForestClassifier(max_depth= 38, max_features='sqrt', min_samples_leaf=0.0022712801155358453,
                             min_samples_split=0.009889030194338, n_estimators=53
                             ,random_state= 1).fit(x_train_greece, y_train_greece)

y_pre_proba_rfc=rfc.predict_proba(x_test_greece)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(pd.Series(y_pre_proba_rfc[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc,y_test_greece),3))
print()
print("Test F1-score",round(f1_score(y_pre_rfc,y_test_greece),3))

Random Forest Classifier

Test BS: 0.158

Test Accuracy: 0.758

Test F1-score 0.828


In [14]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),greece_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Panathinaikos,0.288102
1,Olympiacos,0.276805
2,AEK,0.043196
3,Trikala,0.039605
4,Panelefsiniakos,0.032609
5,Aris,0.031472
6,PAOK,0.030444
7,Koroivos,0.029157
8,Panionios,0.026216
9,Doxa Lefkadas,0.024641


## xgboost

In [15]:
# we have a small sample for training we exclude the subsample hyperparameter

alg = XGBClassifier(colsample_bytree = 0.7120387144249468, gamma = 1.173482282705095, learning_rate = 0.18114216198386113,
                    max_depth= 41, n_estimators= 88, reg_lambda= 0.33962213881868086,
                    objective='binary:logistic').fit(x_train_greece, y_train_greece)




y_pre_proba_alg=alg.predict_proba(x_test_greece)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(pd.Series(y_pre_proba_alg[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg,y_test_greece),3))
print()
print("Test F1-score",round(f1_score(y_pre_alg,y_test_greece),3))


xgboost

Test BS: 0.154

Test Accuracy: 0.78

Test F1-score 0.841


In [16]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),greece_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Panathinaikos,0.191464
1,Olympiacos,0.185551
2,AEK,0.057037
3,Doxa Lefkadas,0.055093
4,Aris,0.037054
5,Panelefsiniakos,0.036969
6,PAOK,0.035535
7,Ikaros,0.034956
8,Ilisiakos,0.034897
9,Trikala,0.034238


# Ensemble

In [17]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_greece),3))
print()
print("Test F1-score",round(f1_score(y_pre_ens,y_test_greece),3))


------Ensemble 3 classifier-------

Test BS: 0.152

Test Accuracy: 0.791

Test F1-score 0.85


# Liga ACB

In [18]:
spain_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Liga ACB'].reset_index(drop = True)
spain_baseline_data['Phase'].unique()

array(['Regular Season', 'Playoffs'], dtype=object)

In [19]:
spain_home = pd.get_dummies(spain_baseline_data["Home Team"]).reset_index(drop = True)
spain_away = -1*pd.get_dummies(spain_baseline_data["Away Team"]).reset_index(drop = True)
spain_x = pd.concat([spain_home,spain_away],axis=1).groupby(level=0, axis=1).sum()
spain_y = spain_baseline_data["Team Result"]

In [20]:
spain_x.head()

,Andorra,Barcelona,Baskonia,Bilbao,Burgos,Estudiantes,Fuenlabrada,Gipuzkoa,Gran Canaria,Joventut,Manresa,Murcia,Obradoiro,Real Betis,Real Madrid,Tenerife,Unicaja Malaga,Valencia,Valladolid,Zaragoza
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0


In [21]:
x_structure_spain = spain_x.iloc[:len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_spain = spain_y.iloc[:len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])]

start = len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])
end = len(pd.concat([spain_baseline_data[(spain_baseline_data["Match Date"]<"2017-8-1")],
                     spain_baseline_data[(spain_baseline_data["Match Date"]>"2017-8-1")&(
                         spain_baseline_data['Phase'].isin(['Regular Season']))]]))

x_spain_17_18 = spain_x.iloc[start:end]
y_spain_17_18 = spain_y.iloc[start:end]

x_train_spain = spain_x[:start + int(len(x_spain_17_18)/2)]
y_train_spain = spain_y.iloc[:start + int(len(x_spain_17_18)/2)]

x_test_spain = x_spain_17_18[int(len(x_spain_17_18)/2):]
y_test_spain = y_spain_17_18.iloc[int(len(y_spain_17_18)/2):]

###  Logistic Regression

In [22]:
lgr = LogisticRegression(solver='liblinear',C = 0.7331693075719703, penalty = 'l1').fit(x_train_spain, y_train_spain)
y_pre_proba_lgr=lgr.predict_proba(x_test_spain)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_spain),3))
print()
print("Test F1-score",round(f1_score(y_pre_lgr,y_test_spain),3))


Logistic Regression

Test BS: 0.202

Test Accuracy: 0.699

Test F1-score 0.775


In [23]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],spain_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()

In [24]:
sorted_coef2

,index,0
0,Real Madrid,2.230753
1,Valladolid,-1.846187
2,Barcelona,1.661323
3,Valencia,1.600437
4,Unicaja Malaga,1.080171
5,Baskonia,0.906135
6,Manresa,-0.841652
7,Gran Canaria,0.783291
8,Tenerife,0.406979
9,Gipuzkoa,-0.369501


In [25]:
lgr.intercept_

array([0.65322085])

## Random Forest Classifier

In [26]:
rfc = RandomForestClassifier(max_depth= 39, max_features='sqrt', min_samples_leaf=0.0009330471732301171,
                             min_samples_split=0.075312968832076, n_estimators=73,
                             random_state=1).fit(x_train_spain, y_train_spain)

y_pre_proba_rfc=rfc.predict_proba(x_test_spain)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(pd.Series(y_pre_proba_rfc[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc,y_test_spain),3))
print()
print("Test F1-score",round(f1_score(y_pre_rfc,y_test_spain),3))

Random Forest Classifier

Test BS: 0.208

Test Accuracy: 0.673

Test F1-score 0.747


In [27]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),spain_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Real Madrid,0.234425
1,Valencia,0.125406
2,Barcelona,0.110825
3,Manresa,0.097176
4,Valladolid,0.056187
5,Unicaja Malaga,0.055950
6,Gipuzkoa,0.050351
7,Baskonia,0.037297
8,Real Betis,0.033387
9,Gran Canaria,0.028629


## xgboost

In [28]:
alg = XGBClassifier(colsample_bytree= 0.6188949028768943, gamma= 2.819830261508139, learning_rate= 0.36256801131990146,
                    max_depth= 21, n_estimators= 86, reg_lambda= 0.09211406347716067, subsample= 0.5079441274104476,
                    objective='binary:logistic').fit(x_train_spain, y_train_spain)

y_pre_proba_alg=alg.predict_proba(x_test_spain)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(pd.Series(y_pre_proba_alg[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg,y_test_spain),3))
print()
print("Test F1-score",round(f1_score(y_pre_alg,y_test_spain),3))


xgboost

Test BS: 0.211

Test Accuracy: 0.68

Test F1-score 0.749


In [29]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),spain_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Real Madrid,0.084033
1,Barcelona,0.079997
2,Manresa,0.070064
3,Valencia,0.059136
4,Unicaja Malaga,0.056946
5,Obradoiro,0.055759
6,Estudiantes,0.053743
7,Burgos,0.048901
8,Murcia,0.047436
9,Gipuzkoa,0.047030


In [30]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_spain),3))
print()
print("Test F1-score",round(f1_score(y_pre_ens,y_test_spain),3))


------Ensemble 3 classifier-------

Test BS: 0.205

Test Accuracy: 0.699

Test F1-score 0.772


# Euroleague

In [31]:
el_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Euroleague'].reset_index(drop = True)
el_baseline_data['Phase'].unique()

array(['Regular Season', 'Top 16', 'Playoffs', 'Final Four'], dtype=object)

In [32]:
el_home = pd.get_dummies(el_baseline_data["Home Team"]).reset_index(drop = True)
el_away = -1*pd.get_dummies(el_baseline_data["Away Team"]).reset_index(drop = True)
el_x = pd.concat([el_home,el_away],axis=1).groupby(level=0, axis=1).sum()
el_y = el_baseline_data["Team Result"]

In [33]:
el_x.head()

,Alba Berlin,Anadolu Efes,Armani Milano,Bamberg,Barcelona,Baskonia,Bayern Munich,Budivelnyk,CSKA Moscow,Cedevita,Crvena Zvezda,Darussafaka,Dinamo Sassari,Fenerbahce,Galatasaray,Khimki,Limoges,Lokomotiv Kuban,Maccabi Tel Aviv,Nanterre,Neptunas,Nizhny Novgorod,Olympiacos,Panathinaikos,Partizan,Pinar Karsiyaka,Real Madrid,Rytas Vilnius,Siena,Strasbourg,Turow Zgorzelec,Unicaja Malaga,Unics Kazan,Valencia,Zalgiris,Zielona Gora
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
x_structure_el = el_x.iloc[:len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_el = el_y.iloc[:len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])]

start = len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])
end = len(pd.concat([el_baseline_data[(el_baseline_data["Match Date"]<"2017-8-1")],
                     el_baseline_data[(el_baseline_data["Match Date"]>"2017-8-1")&(
                         el_baseline_data['Phase'].isin(['Regular Season','Top 16']))]]))

x_el_17_18 = el_x.iloc[start:end]
y_el_17_18 = el_y.iloc[start:end]

x_train_el = el_x[:start + int(len(x_el_17_18)/2)]
y_train_el = el_y.iloc[:start + int(len(x_el_17_18)/2)]

x_test_el = x_el_17_18[int(len(x_el_17_18)/2):]
y_test_el = y_el_17_18.iloc[int(len(y_el_17_18)/2):]

###  Logistic Regression

In [35]:
lgr = LogisticRegression(solver='liblinear' , C= 0.7847137962592965, penalty='l1').fit(x_train_el, y_train_el)
y_pre_proba_lgr=lgr.predict_proba(x_test_el)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_el),3))
print()
print("Test F1-score",round(f1_score(y_pre_lgr,y_test_el),3))


Logistic Regression

Test BS: 0.204

Test Accuracy: 0.658

Test F1-score 0.754


In [36]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],el_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()

In [37]:
sorted_coef3.head(50)

,index,0
0,Dinamo Sassari,-1.923382
1,CSKA Moscow,1.886007
2,Fenerbahce,1.390589
3,Real Madrid,1.368333
4,Olympiacos,1.291038
5,Barcelona,0.930188
6,Lokomotiv Kuban,0.906466
7,Turow Zgorzelec,-0.826674
8,Zielona Gora,-0.821117
9,Panathinaikos,0.734394


In [38]:
lgr.intercept_

array([0.6163098])

## Random Forest Classifier

In [39]:
rfc = RandomForestClassifier(max_depth= 36, max_features='log2', min_samples_leaf=0.002390948900012013,
                             min_samples_split=0.0096605683266830, n_estimators=73,
                             random_state=1).fit(x_train_el, y_train_el)

y_pre_proba_rfc=rfc.predict_proba(x_test_el)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(y_pre_proba_rfc[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc ,y_test_el),3))
print()
print("Test F1-score",round(f1_score(y_pre_rfc,y_test_el),3))

Random Forest Classifier

Test BS: 0.206

Test Accuracy: 0.667

Test F1-score 0.773


In [40]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),el_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,CSKA Moscow,0.184488
1,Fenerbahce,0.092523
2,Real Madrid,0.076838
3,Olympiacos,0.069640
4,Dinamo Sassari,0.055758
5,Cedevita,0.037479
6,Barcelona,0.033997
7,Zielona Gora,0.032932
8,Panathinaikos,0.031237
9,Armani Milano,0.028539


## xgboost

In [41]:
alg = XGBClassifier(colsample_bytree= 0.897167919370632, gamma= 0.835671500072126, learning_rate= 0.3887154858505634,
                    max_depth= 1, n_estimators= 99, reg_lambda= 0.7909237442174416, subsample= 0.963659261900305,
                    objective='binary:logistic').fit(x_train_el, y_train_el)

y_pre_proba_alg=alg.predict_proba(x_test_el)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(y_pre_proba_alg[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg ,y_test_el),3))
print()
print("Test F1-score",round(f1_score(y_pre_alg,y_test_el),3))


xgboost

Test BS: 0.207

Test Accuracy: 0.667

Test F1-score 0.775


In [42]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),el_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,CSKA Moscow,0.090483
1,Fenerbahce,0.069650
2,Cedevita,0.065805
3,Olympiacos,0.061985
4,Real Madrid,0.061631
5,Panathinaikos,0.050993
6,Zielona Gora,0.045661
7,Maccabi Tel Aviv,0.042520
8,Dinamo Sassari,0.041238
9,Barcelona,0.040985


In [43]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test RPS:",round(brier_score_loss(y_test_el,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_el),3))
print()
print("Test F1-score",round(f1_score(y_pre_ens,y_test_el),3))


------Ensemble 3 classifier-------

Test RPS: 0.204

Test Accuracy: 0.65

Test F1-score 0.756


# Eurocup

In [44]:
ec_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Eurocup'].reset_index(drop = True)
ec_baseline_data['Phase'].unique()

array(['Regular Season', 'Last 32', 'Quarter-Finals', 'Semifinals',
       'Finals', 'Top 16'], dtype=object)

In [45]:
ec_baseline_data= ec_baseline_data[ec_baseline_data['Phase'].isin(['Regular Season','Last 32','Top 16'])]
ec_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Eurocup,2013-10-15,Bonn,Alba Berlin,0.0,Alba Berlin,Regular Season
1,Eurocup,2013-10-15,Banvit,Kalev Cramo,1.0,Banvit,Regular Season
2,Eurocup,2013-10-15,PAOK,Buducnost,1.0,PAOK,Regular Season
3,Eurocup,2013-10-15,Bilbao,Cedevita,1.0,Bilbao,Regular Season
4,Eurocup,2013-10-15,Paris Levallois,Valencia,0.0,Valencia,Regular Season


In [46]:
ec_home = pd.get_dummies(ec_baseline_data["Home Team"]).reset_index(drop = True)
ec_away = -1*pd.get_dummies(ec_baseline_data["Away Team"]).reset_index(drop = True)
ec_x = pd.concat([ec_home,ec_away],axis=1).groupby(level=0, axis=1).sum()
ec_y = ec_baseline_data["Team Result"]

In [47]:
ec_x.head()

,AEK,ASVEL Villeurbanne,Alba Berlin,Alba Fehervar,Andorra,Aris,Armani Milano,Artland Dragons,Avtodor Saratov,Bamberg,Banvit,Bayern Munich,Besiktas,Bilbao,Bisons Loimaa,Bonn,Brindisi,Budivelnyk,Buducnost,Cantu,Cedevita,Chalon Saone,Cibona,Crvena Zvezda,Csu Asesoft Ploiesti,Darussafaka,Dijon,Dinamo Sassari,Fuenlabrada,Galatasaray,Gran Canaria,Gravelines,Hapoel Jerusalem,Igokea,Kalev Cramo,Khimik,Khimki,Krasny Oktyabr,Le Mans,Lietkabelis,Limoges,Lokomotiv Kuban,Ludwigsburg,Lukoil Academic,MZT Skopje,Maccabi Haifa,Maccabi Tel Aviv,Mons,Murcia,Nancy,Nanterre,Neptunas,Nizhny Novgorod,Nymburk,Oldenburg,Olimpija Ljubljana,Oostende,PAOK,Panionios,Paris Levallois,Partizan,Pinar Karsiyaka,Radnicki,Real Betis,Reggio Emilia,Rytas Vilnius,Siena,Spartak St Petersburg,Spirou Charleroi,Steaua Bucharest,Strasbourg,Szolnoki,Ted Ankara Kolejliler,Tofas Bursa,Trabzonspor,Trento,Turin,Turow Zgorzelec,Ulm,Unicaja Malaga,Unics Kazan,VEF Riga,Valencia,Varese,Venezia,Ventspils,Virtus Roma,Zaragoza,Zenit St Petersburg,Zielona Gora
0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0


In [48]:
x_structure_ec = ec_x.iloc[:len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_ec = ec_y.iloc[:len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])]

start = len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])
end = len(pd.concat([ec_baseline_data[(ec_baseline_data["Match Date"]<"2017-8-1")],
                     ec_baseline_data[(ec_baseline_data["Match Date"]>"2017-8-1")&(
                         ec_baseline_data['Phase'].isin(['Regular Season','Last 32','Top 16']))]]))

x_ec_17_18 = ec_x.iloc[start:end]
y_ec_17_18 = ec_y.iloc[start:end]

x_train_ec = ec_x[:start + int(len(x_ec_17_18)/2)]
y_train_ec = ec_y.iloc[:start + int(len(x_ec_17_18)/2)]

x_test_ec = x_ec_17_18[int(len(x_ec_17_18)/2):]
y_test_ec = y_ec_17_18.iloc[int(len(y_ec_17_18)/2):]

###  Logistic Regression

In [49]:
lgr = LogisticRegression(solver='liblinear',C= 0.8479198446641489, penalty= 'l1').fit(x_train_ec, y_train_ec)
y_pre_proba_lgr=lgr.predict_proba(x_test_ec)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_ec),3))
print()
print("Test F1-score",round(f1_score(y_pre_lgr,y_test_ec),3))


Logistic Regression

Test BS: 0.219

Test Accuracy: 0.702

Test F1-score 0.762


In [50]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],ec_x.columns.values):
    coef_dict[feat] = coef
coef4=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef4=coef4.iloc[(-coef4[0].abs()).argsort()].reset_index()

In [51]:
sorted_coef4.head(50)

,index,0
0,MZT Skopje,-1.976727
1,Bayern Munich,1.959183
2,Cibona,-1.861851
3,Lokomotiv Kuban,1.710876
4,Khimki,1.634689
5,Artland Dragons,-1.631817
6,Unics Kazan,1.521564
7,Gran Canaria,1.493483
8,Brindisi,-1.170797
9,Valencia,1.088403


In [52]:
lgr.intercept_

array([0.54356355])

## Random Forest Classifier

In [53]:
rfc = RandomForestClassifier(max_depth= 38, max_features='sqrt', min_samples_leaf=0.0037012163422379674,
                             min_samples_split=0.251132570859313, n_estimators=41,
                             random_state=1).fit(x_train_ec, y_train_ec)

y_pre_proba_rfc=rfc.predict_proba(x_test_ec)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(y_pre_proba_rfc[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc ,y_test_ec),3))
print()
print("Test F1-score",round(f1_score(y_pre_rfc,y_test_ec),3))

Random Forest Classifier

Test BS: 0.24

Test Accuracy: 0.643

Test F1-score 0.737


In [55]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),ec_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Bayern Munich,0.079578
1,Lokomotiv Kuban,0.071354
2,Khimki,0.070220
3,Gran Canaria,0.068680
4,Unics Kazan,0.065164
...,...,...
85,Crvena Zvezda,0.000000
86,Chalon Saone,0.000000
87,Budivelnyk,0.000000
88,Tofas Bursa,0.000000


## xgboost

In [56]:
alg = XGBClassifier(colsample_bytree= 0.5913621206231853, gamma= 4.08026456094463, learning_rate= 0.37903160103981465,
                    max_depth= 47, n_estimators= 94, reg_lambda= 0.11419103539605524,
                    objective='binary:logistic').fit(x_train_ec, y_train_ec)

y_pre_proba_alg=alg.predict_proba(x_test_ec)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(y_pre_proba_alg[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg ,y_test_ec),3))
print()
print("Test F1-score",round(f1_score(y_pre_alg,y_test_ec),3))


xgboost

Test BS: 0.229

Test Accuracy: 0.643

Test F1-score 0.737


In [57]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),ec_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Bayern Munich,0.065245
1,Lokomotiv Kuban,0.057316
2,Valencia,0.056947
3,Artland Dragons,0.055598
4,Cibona,0.055272
...,...,...
85,Khimik,0.000000
86,Kalev Cramo,0.000000
87,Igokea,0.000000
88,Alba Fehervar,0.000000


In [58]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_ec),3))
print()
print("Test F1-score",round(f1_score(y_pre_ens,y_test_ec),3))


------Ensemble 3 classifier-------

Test BS: 0.226

Test Accuracy: 0.643

Test F1-score 0.737
